## Load Package

In [ ]:
#! pip install pylibjpeg pylibjpeg-libjpeg pydicom
#! pip install -U python-gdcm

In [ ]:
#!python -m pip download python-gdcm -q
#!python -m pip download pylibjpeg-libjpeg -q
#!python -m pip download pylibjpeg -q

In [1]:
import sys
sys.path.append('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\kerasapplications')
sys.path.append('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\efficientnet-keras-source-code\\')
import keras_applications
import efficientnet.tfkeras as efficientnet

C:\Users\public.DESKTOP-P7554KD\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2
import os
from os import listdir
import re
import gc
from time import time
from datetime import datetime

import gdcm
import pydicom as dicom
from pydicom import dcmread
import pylibjpeg
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom.data import get_testdata_files

from path import Path
import scipy.ndimage
from tqdm import tqdm
import nibabel as nib
from pprint import pprint
from time import time
import itertools
from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from glob import glob
import warnings
import dask.array as da

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, cohen_kappa_score, confusion_matrix
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses, callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub
from keras.preprocessing.image import ImageDataGenerator
from keras import layers

from PIL import Image as im
import traceback

import imageio


#from efficientnet_v2 import EfficientNetV2S
%matplotlib inline
sns.set(style='darkgrid', font_scale=1.6)

## Get Training Images and Training Lables

In [3]:

train_df = pd.read_csv("C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\train.csv")
test_df = pd.read_csv("C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\test.csv")

train_dir = 'C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\train_images'
test_dir = 'C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\test_images'


In [ ]:
train_df.head()

In [9]:
train_df.iloc[0]

StudyInstanceUID    1.2.826.0.1.3680043.6200
patient_overall                            1
C1                                         1
C2                                         1
C3                                         0
C4                                         0
C5                                         0
C6                                         0
C7                                         0
Name: 0, dtype: object

In [5]:
test_df

,row_id,StudyInstanceUID,prediction_type
0,1.2.826.0.1.3680043.22327_C1,1.2.826.0.1.3680043.22327,C1
1,1.2.826.0.1.3680043.25399_C1,1.2.826.0.1.3680043.25399,C1
2,1.2.826.0.1.3680043.5876_C1,1.2.826.0.1.3680043.5876,C1


## Reformat the training dataset

In [4]:
pred_df = pd.DataFrame(columns = ['actual','row_id', 'StudyInstanceUID','prediction_type'])
label = []
row_id = []
StudyInstanceUID = []
prediction_type = []

for i in range(len(train_df)):

    StudyInstanceUID.append(train_df.iloc[i]['StudyInstanceUID'])
    label.append(train_df.iloc[i]['patient_overall'])
    row_id.append(train_df.iloc[i]['StudyInstanceUID']+'_'+'patient_overall')
    prediction_type.append('patient_overall')

    StudyInstanceUID.append(train_df.iloc[i]['StudyInstanceUID'])
    label.append(train_df.iloc[i]['C1'])
    row_id.append(train_df.iloc[i]['StudyInstanceUID']+'_'+'C1')
    prediction_type.append('C1')

    StudyInstanceUID.append(train_df.iloc[i]['StudyInstanceUID'])
    label.append(train_df.iloc[i]['C2'])
    row_id.append(train_df.iloc[i]['StudyInstanceUID']+'_'+'C2')
    prediction_type.append('C2')

    StudyInstanceUID.append(train_df.iloc[i]['StudyInstanceUID'])
    label.append(train_df.iloc[i]['C3'])
    row_id.append(train_df.iloc[i]['StudyInstanceUID']+'_'+'C3')
    prediction_type.append('C3')

    StudyInstanceUID.append(train_df.iloc[i]['StudyInstanceUID'])
    label.append(train_df.iloc[i]['C4'])
    row_id.append(train_df.iloc[i]['StudyInstanceUID']+'_'+'C4')
    prediction_type.append('C4')

    StudyInstanceUID.append(train_df.iloc[i]['StudyInstanceUID'])
    label.append(train_df.iloc[i]['C5'])
    row_id.append(train_df.iloc[i]['StudyInstanceUID']+'_'+'C5')
    prediction_type.append('C5')

    StudyInstanceUID.append(train_df.iloc[i]['StudyInstanceUID'])
    label.append(train_df.iloc[i]['C6'])
    row_id.append(train_df.iloc[i]['StudyInstanceUID']+'_'+'C6')
    prediction_type.append('C6')

    StudyInstanceUID.append(train_df.iloc[i]['StudyInstanceUID'])
    label.append(train_df.iloc[i]['C7'])
    row_id.append(train_df.iloc[i]['StudyInstanceUID']+'_'+'C7')
    prediction_type.append('C7')

pred_df['actual'] = label
pred_df['row_id'] = row_id
pred_df['StudyInstanceUID'] = StudyInstanceUID
pred_df['prediction_type'] = prediction_type
pred_df['fractured'] = 0
pred_df['inference_time'] = 0
pred_df

,actual,row_id,StudyInstanceUID,prediction_type,fractured,inference_time
0,1,1.2.826.0.1.3680043.6200_patient_overall,1.2.826.0.1.3680043.6200,patient_overall,0,0
1,1,1.2.826.0.1.3680043.6200_C1,1.2.826.0.1.3680043.6200,C1,0,0
2,1,1.2.826.0.1.3680043.6200_C2,1.2.826.0.1.3680043.6200,C2,0,0
3,0,1.2.826.0.1.3680043.6200_C3,1.2.826.0.1.3680043.6200,C3,0,0
4,0,1.2.826.0.1.3680043.6200_C4,1.2.826.0.1.3680043.6200,C4,0,0
...,...,...,...,...,...,...
16147,0,1.2.826.0.1.3680043.18786_C3,1.2.826.0.1.3680043.18786,C3,0,0
16148,0,1.2.826.0.1.3680043.18786_C4,1.2.826.0.1.3680043.18786,C4,0,0
16149,0,1.2.826.0.1.3680043.18786_C5,1.2.826.0.1.3680043.18786,C5,0,0
16150,0,1.2.826.0.1.3680043.18786_C6,1.2.826.0.1.3680043.18786,C6,0,0


## Read Image

In [5]:
#Loading Scans
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

# Load the scans in given folder path
def load_scan(path):
    
    dcm_paths = glob(f"{path}/*")
    dcm_paths.sort(key=natural_keys)
    
    patient_scan = [dicom.dcmread(paths) for paths in dcm_paths]
    
    return patient_scan

## HU Rescaling

In [6]:
def get_pixels_hu(slices):
   
    image = np.stack([cv2.resize(s.pixel_array,(256,256),interpolation = cv2.INTER_NEAREST) for s in slices])
    #image = np.stack([cv2.cvtColor(s.pixel_array.reshape(512,512),cv2.COLOR_GRAY2RGB) for s in slices])

    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16) #int16
    image = da.from_array(image) #Using Dask to speed up processing
    
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image <= -1000] = 0
    
    # Convert to Hounsfield units (HU)
        
    intercept = da.from_array([slices[slice_number].RescaleIntercept for slice_number in range(len(slices))])
    slope = da.from_array([slices[slice_number].RescaleSlope for slice_number in range(len(slices))])
    
    intercept=intercept.reshape((-1,1,1))
    slope=slope.reshape((-1,1,1))
    #print(slope.shape)
    #print(image.shape)
    image= slope * image.astype("float64")
    #print(image.shape)   
    image+= intercept
    #print(image.shape)      
    return image.astype("int16")

## Normalization

In [7]:
MIN_BOUND = 150.0
MAX_BOUND = 2050.0
    
def normalize(image):
    image = (image - MIN_BOUND)*255.0 / (MAX_BOUND - MIN_BOUND)
    image[image>255] = 255.
    image[image<0] = 255.
    
    image = image.astype(np.int16)
    return image

## Load Model

In [8]:
model_whole_c1 = keras.models.load_model('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\effnet_whole_c1_wo_bias2.h5')
model_whole_c2 = keras.models.load_model('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\effnet_whole_c2_wo_bias2.h5')
model_whole_c3 = keras.models.load_model('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\effnet_whole_c3_wo_bias2.h5')
model_whole_c4 = keras.models.load_model('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\effnet_whole_c4_wo_bias2.h5')
model_whole_c5 = keras.models.load_model('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\effnet_whole_c5_wo_bias2.h5')
model_whole_c6 = keras.models.load_model('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\effnet_whole_c6_wo_bias2.h5')
model_whole_c7 = keras.models.load_model('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\effnet_whole_c7_wo_bias2.h5')
model_whole_overall = keras.models.load_model('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\effnet_whole_overall_wo_bias2.h5')

## Inference Data Generator

In [9]:
def RSNATestGenerator(test_df, batch_size, infinite = True, base_path = train_dir):
    while 1:        
        testset=[]
        testidt=[]
      
        idt = test_df['StudyInstanceUID']
        path = os.path.join(base_path, idt)
            
        patient_scan=load_scan(path)
        patient_hu=get_pixels_hu(patient_scan)
        patient_hu_normalised=normalize(patient_hu)
            
        if os.path.exists(path):
            for j in range(len(os.listdir(path))):
                img = patient_hu_normalised[j]
                img = np.array(img)
                img=cv2.resize(img,(128, 128))
                image=img_to_array(img)
                image=image/255.0
                testset+=[image]
                testidt+=[idt]
                if len(testset) == batch_size:                        
                    yield np.array(testset)
                    testset = []


## Inference

In [11]:
for i in range(50):
    start = 450
    start_time = time()
    
    # predict C1 to C7 and overall

    preds_c1 = model_whole_c1.predict(RSNATestGenerator(train_df.iloc[start+i], 1, 
                                                      infinite = False, base_path = train_dir),steps = 1)
    preds_c2 = model_whole_c2.predict(RSNATestGenerator(train_df.iloc[start+i], 1, 
                                                      infinite = False, base_path = train_dir),steps = 1)
    preds_c3 = model_whole_c3.predict(RSNATestGenerator(train_df.iloc[start+i], 1, 
                                                      infinite = False, base_path = train_dir),steps = 1)
    preds_c4 = model_whole_c4.predict(RSNATestGenerator(train_df.iloc[start+i], 1, 
                                                      infinite = False, base_path = train_dir),steps = 1)
    preds_c5 = model_whole_c5.predict(RSNATestGenerator(train_df.iloc[start+i], 1, 
                                                      infinite = False, base_path = train_dir),steps = 1)
    preds_c6 = model_whole_c6.predict(RSNATestGenerator(train_df.iloc[start+i], 1, 
                                                      infinite = False, base_path = train_dir),steps = 1)
    preds_c7 = model_whole_c7.predict(RSNATestGenerator(train_df.iloc[start+i], 1, 
                                                      infinite = False, base_path = train_dir),steps = 1)
    preds_overall = model_whole_overall.predict(RSNATestGenerator(train_df.iloc[start+i], 1, 
                                                      infinite = False, base_path = train_dir),steps = 1)
    
    end_time = time()
    delta = (end_time-start_time) #in seconds
    
    # update time
    pred_df.loc[pred_df["StudyInstanceUID"] == train_df.iloc[start+i]['StudyInstanceUID'], "inference_time"] = delta

    # update inference dataframe
    pred_df.loc[(pred_df["StudyInstanceUID"] == train_df.iloc[start+i]['StudyInstanceUID']) 
                & (pred_df["prediction_type"] == 'C1'), "fractured"] = preds_c1[0][0]
    pred_df.loc[(pred_df["StudyInstanceUID"] == train_df.iloc[start+i]['StudyInstanceUID']) 
                & (pred_df["prediction_type"] == 'C2'),  "fractured"] = preds_c2[0][0]
    pred_df.loc[(pred_df["StudyInstanceUID"] == train_df.iloc[start+i]['StudyInstanceUID']) 
                & (pred_df["prediction_type"] == 'C3'), "fractured"] = preds_c3[0][0]
    pred_df.loc[(pred_df["StudyInstanceUID"] == train_df.iloc[start+i]['StudyInstanceUID']) 
                & (pred_df["prediction_type"] == 'C4'), "fractured"] = preds_c4[0][0]
    pred_df.loc[(pred_df["StudyInstanceUID"] == train_df.iloc[start+i]['StudyInstanceUID']) 
                & (pred_df["prediction_type"] == 'C5'), "fractured"] = preds_c5[0][0]
    pred_df.loc[(pred_df["StudyInstanceUID"] == train_df.iloc[start+i]['StudyInstanceUID']) 
                & (pred_df["prediction_type"] == 'C6'), "fractured"] = preds_c6[0][0]
    pred_df.loc[(pred_df["StudyInstanceUID"] == train_df.iloc[start+i]['StudyInstanceUID']) 
                & (pred_df["prediction_type"] == 'C7'), "fractured"] = preds_c7[0][0]
    pred_df.loc[(pred_df["StudyInstanceUID"] == train_df.iloc[start+i]['StudyInstanceUID']) 
                & (pred_df["prediction_type"] == 'patient_overall'), "fractured"] = preds_overall[0][0]

1/1 [==============================] - 0s 187ms/step


1/1 [==============================] - 1s 508ms/step


1/1 [==============================] - 1s 652ms/step


In [16]:
pred_df.iloc[800:1600]

,actual,row_id,StudyInstanceUID,prediction_type,fractured,inference_time
800,0,1.2.826.0.1.3680043.18696_patient_overall,1.2.826.0.1.3680043.18696,patient_overall,0.000000,8.070511
801,0,1.2.826.0.1.3680043.18696_C1,1.2.826.0.1.3680043.18696,C1,0.000000,8.070511
802,0,1.2.826.0.1.3680043.18696_C2,1.2.826.0.1.3680043.18696,C2,0.121009,8.070511
803,0,1.2.826.0.1.3680043.18696_C3,1.2.826.0.1.3680043.18696,C3,0.000000,8.070511
804,0,1.2.826.0.1.3680043.18696_C4,1.2.826.0.1.3680043.18696,C4,0.000000,8.070511
...,...,...,...,...,...,...
1595,0,1.2.826.0.1.3680043.4041_C3,1.2.826.0.1.3680043.4041,C3,0.000000,11.668661
1596,0,1.2.826.0.1.3680043.4041_C4,1.2.826.0.1.3680043.4041,C4,0.000000,11.668661
1597,0,1.2.826.0.1.3680043.4041_C5,1.2.826.0.1.3680043.4041,C5,0.000000,11.668661
1598,0,1.2.826.0.1.3680043.4041_C6,1.2.826.0.1.3680043.4041,C6,0.000000,11.668661


In [12]:
pred_df.to_csv('pred_df_weighted_450_500.csv', index=False)

In [23]:
pred_df_read = pd.read_csv('pred_df_weighted.csv')

In [24]:
pred_df['fractured_prob'] = pred_df['fractured']

In [25]:
pred_df.loc[(pred_df["fractured_prob"] > 0.40) & pred_df["prediction_type"] == 'patient_overall', "fractured"] = 1
pred_df.loc[(pred_df["fractured_prob"] <= 0.40) & pred_df["prediction_type"] == 'patient_overall', "fractured"] = 0
pred_df.loc[(pred_df["fractured_prob"] > 0.16) & pred_df["prediction_type"] != 'patient_overall', "fractured"] = 1
pred_df.loc[(pred_df["fractured_prob"] <= 0.16) & pred_df["prediction_type"] != 'patient_overall', "fractured"] = 0


## Error Analysis

In [14]:
def define_eval_metrics(df,verbose = False):
    cls_report = classification_report(y_true=df.actual, y_pred = df.fractured, zero_division=0, output_dict=True)

    recall = cls_report['macro avg']['recall']
    precision = cls_report['macro avg']['precision']

    tn, fp, fn, tp = confusion_matrix(y_true=df.actual, y_pred = df.fractured).ravel()

    fpr = fp / (fp+tn)
    fnr = fn / (fn+tn)

    pred_pos_rate = np.mean(df.fractured == 1)
    actual_pos_rate = np.mean(df.actual == 1)

    if verbose:

        print('\nClassification Report')
        pprint(cls_report)

        print(f"""
        TN: {tn}
        FP: {fp}
        FN: {fn}
        TP: {tp}
        """)

        # print FPR and FNR
        print(f"FPR: {fpr :.4}")
        print(f"FNR: {fnr :.4}")

        # value count + value count normalized for predictions.
        print('Number of fractured vs non-fractured predictions')
        print(df.fractured.value_counts())
        print('Ratio of fractured vs non-fractured predictions')
        print(df.fractured.value_counts(normalize=True))

        # What is the predicted positive fracture rate compared to the actual?
        print(f"""
        Predicted positive fracture rate: {np.round(pred_pos_rate,4)}
        Actual fracture rate {np.round(actual_pos_rate,4)}"""
        )
    
    return([recall,precision,tn,fp,fn,tp,fpr,fnr])

def eval_model(df):
    
    try:
        print(f'Average inference time : {inference_time/(len(df)/8)} s per patient')
    except:
        print('No time data available')

    # initialize patient and vertebrae df
    df['fractured'] = round(df.fractured)
    patient_df = df[df.row_id.str.contains("patient_overall")]
    vert_df =  df[~df.row_id.str.contains("patient_overall")]

    # store eval metrics
    eval_metrics = dict()
    eval_metrics['eval_metric'] = ['recall','precision','tn','fp','fn','tp','fpr','fnr']

    print(f"{'*' * 18} Evaluating model at patient-level {'*' * 18}")
    eval_metrics['patient_level'] = define_eval_metrics(patient_df)

    print(f"{'*' * 18} Evaluating model at vertebrae-level {'*' * 18}")
    eval_metrics['vertebrae_level'] = define_eval_metrics(vert_df)
    
    # which vertebrae are missed the most? FP FN?
    vert_df.loc[:,'vertebrae'] = vert_df.row_id.str.split('_',).apply(lambda x: x[-1])

    for vertebra in vert_df.vertebrae.unique():
        print(f"{'*'*18} {vertebra} {'*'*18}")
        eval_metrics[vertebra] = define_eval_metrics(
            vert_df.loc[vert_df.vertebrae == vertebra])

    
    # issues with patient overall not match patient vertebrae prediction?
    
    return(pd.DataFrame.from_dict(eval_metrics))

In [15]:
eval_model(pred_df.iloc[:1600])

No time data available


C:\Users\public.DESKTOP-P7554KD\AppData\Local\Temp\ipykernel_8904\997355233.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fractured'] = round(df.fractured)


****************** Evaluating model at patient-level ******************
****************** Evaluating model at vertebrae-level ******************
****************** C1 ******************
****************** C2 ******************
****************** C3 ******************
****************** C4 ******************
****************** C5 ******************
****************** C6 ******************
****************** C7 ******************


C:\Users\public.DESKTOP-P7554KD\AppData\Local\Temp\ipykernel_8904\997355233.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vert_df.loc[:,'vertebrae'] = vert_df.row_id.str.split('_',).apply(lambda x: x[-1])


,eval_metric,patient_level,vertebrae_level,C1,C2,C3,C4,C5,C6,C7
0,recall,0.500,0.500000,0.5000,0.50,0.500,0.50,0.500,0.500,0.50
1,precision,0.235,0.441071,0.4725,0.45,0.485,0.46,0.445,0.425,0.35
2,tn,94.000,1235.000000,189.0000,180.00,194.000,184.00,178.000,170.000,140.00
3,fp,0.000,0.000000,0.0000,0.00,0.000,0.00,0.000,0.000,0.00
4,fn,106.000,165.000000,11.0000,20.00,6.000,16.00,22.000,30.000,60.00
5,tp,0.000,0.000000,0.0000,0.00,0.000,0.00,0.000,0.000,0.00
6,fpr,0.000,0.000000,0.0000,0.00,0.000,0.00,0.000,0.000,0.00
7,fnr,0.530,0.117857,0.0550,0.10,0.030,0.08,0.110,0.150,0.30
